<a href="https://colab.research.google.com/github/vkjadon/tf/blob/main/tf203-dogsvscats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import shutil
import glob

import seaborn as sns
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [2]:
!mkdir ~/.kaggle 

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c dogs-vs-cats

100% 810M/812M [00:28<00:00, 32.7MB/s]
100% 812M/812M [00:28<00:00, 29.4MB/s]


In [5]:
!unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [ ]:
!unzip train.zip

In [8]:
!ls

dogs-vs-cats.zip  sample_data		test1.zip  train.zip
kaggle.json	  sampleSubmission.csv	train


In [20]:
if os.path.isdir('train/dog') is False:
  os.makedirs('train/dog')
if os.path.isdir('train/cat') is False:
  os.makedirs('train/cat')
if os.path.isdir('valid/dog') is False:
  os.makedirs('valid/dog')
if os.path.isdir('valid/cat') is False:
  os.makedirs('valid/cat')
if os.path.isdir('test/dog') is False:
  os.makedirs('test/dog')
if os.path.isdir('test/cat') is False:
  os.makedirs('test/cat')

In [19]:
filename=glob.glob('train/cat*')
print(len(filename))

1


In [12]:
for c in random.sample(glob.glob('train/cat*'),11000):
  shutil.move(c,'train/cat')
for c in random.sample(glob.glob('train/dog*'),11000):
  shutil.move(c,'train/dog')

In [18]:
filename=glob.glob('train/cat/*')
print(len(filename))

0


In [ ]:
for c in random.sample(glob.glob('train/cat*'),1000):
  shutil.move(c,'valid/cat')
for c in random.sample(glob.glob('train/dog*'),1000):
  shutil.move(c,'valid/dog')


In [17]:
for c in random.sample(glob.glob('train/cat*'),500):
  shutil.move(c,'test/cat')
for c in random.sample(glob.glob('train/dog*'),500):
  shutil.move(c,'test/dog')